<a href="https://colab.research.google.com/github/nmdimaria/Spotify_Project/blob/will_grace/ML_OverSam_RanFor_actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

file_to_read = pd.read_csv('filtered_data_70.csv')
df = pd.DataFrame(file_to_read)
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,release_date,year,bi_popularity
0,R&B,Rihanna,Desperado,6KFaHC9G178beAp7P0Vi5S,63,0.3230,0.685,186467,0.610,0.000000,C,0.1020,-5.221,Minor,0.0439,94.384,3/4,0.3230,2016-01-28,2016,0
1,R&B,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,6muW8cSjJ3rusKJ0vH5olw,62,0.0675,0.762,199520,0.520,0.000004,F,0.1140,-5.237,Minor,0.0959,75.047,4/4,0.0862,2018-09-21,2018,0
2,R&B,Surfaces,Heaven Falls / Fall on Me,7yHqOZfsXYlicyoMt62yC6,61,0.3600,0.563,240597,0.366,0.002430,B,0.0955,-6.896,Minor,0.1210,85.352,4/4,0.7680,2018-08-24,2018,0
3,R&B,Olivia O'Brien,Love Myself,4XzgjxGKqULifVf7mnDIQK,68,0.5960,0.653,213947,0.621,0.000000,B,0.0811,-5.721,Minor,0.0409,100.006,4/4,0.4660,2019-02-01,2019,0
4,R&B,ELHAE,Needs,7KdRu0h7PQ0Ecfa37rUBzW,61,0.6610,0.510,205640,0.331,0.000000,B,0.1230,-13.073,Minor,0.0895,124.657,4/4,0.2250,2016-03-11,2016,0


In [ ]:
y=df["bi_popularity"]
X=df.drop(columns=["popularity","track_id", "year","bi_popularity", "genre","artist_name", "track_name", "release_date"])

In [ ]:
# Get Dummies for mode and key
X = pd.get_dummies(X, columns=["mode", "key", "time_signature"])
X.head()


,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,mode_Major,mode_Minor,key_A,key_A#,key_B,key_C,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,time_signature_0/4,time_signature_1/4,time_signature_3/4,time_signature_4/4,time_signature_5/4
0,0.3230,0.685,186467,0.610,0.000000,0.1020,-5.221,0.0439,94.384,0.3230,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0.0675,0.762,199520,0.520,0.000004,0.1140,-5.237,0.0959,75.047,0.0862,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2,0.3600,0.563,240597,0.366,0.002430,0.0955,-6.896,0.1210,85.352,0.7680,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0.5960,0.653,213947,0.621,0.000000,0.0811,-5.721,0.0409,100.006,0.4660,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.6610,0.510,205640,0.331,0.000000,0.1230,-13.073,0.0895,124.657,0.2250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
from imblearn.over_sampling import SMOTE
os = SMOTE()
X_oversample, y_oversample = os.fit_resample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#Features shape
X_oversample.shape

(180862, 29)

In [ ]:
# split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_oversample,
   y_oversample,  random_state=1)
X_train.shape

(135646, 29)

In [ ]:
# Scaling data for faster training time
# Creating a StandardScaler instance.
#scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
#X_scaler = scaler.fit(X_train)

# Scaling the data.
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Log Regression model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2,
   random_state=1)

In [ ]:
# Train the model
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
# Create precictions with the model
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,1,1
1,0,0
2,1,1
3,1,1
4,1,0


In [ ]:
# Asses acuraccy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7666312809624911

In [ ]:
# Generate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.67      0.74     22648
           1       0.72      0.86      0.79     22568

    accuracy                           0.77     45216
   macro avg       0.78      0.77      0.76     45216
weighted avg       0.78      0.77      0.76     45216



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[15245  7403]
 [ 3149 19419]]


In [ ]:
results["Actual"].value_counts()

0    22648
1    22568
Name: Actual, dtype: int64

In [ ]:
results["Prediction"].value_counts()

1    26822
0    18394
Name: Prediction, dtype: int64

Save ML Results


In [ ]:
import pickle

Pkl_Filename = "Pickle_RanFor_Model.pkl"  

In [ ]:
#Save the file to working directory
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_RanFor_Model = pickle.load(file)

Pickled_RanFor_Model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = Pickled_RanFor_Model.score(X_test_scaled, y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_RanFor_Model.predict(X_test_scaled)  

Ypredict

NameError: ignored